In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials

import warnings
warnings.filterwarnings('ignore')

In [29]:
df=pd.read_csv('C:\\Users\\pkmra_fk94jjw\\Downloads\\titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [31]:
X=df[['Pclass','SibSp','Parch','Fare']]
y=df.Survived

# SVM model and classification

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [52]:
svc=SVC(kernel='linear', C=1) 
svc.fit(X_train,y_train) 
y_pred=svc.predict(X_test)

In [53]:
print('Accuracy:\n',accuracy_score(y_test, y_pred))
print('Confusion Matrix:\n',confusion_matrix(y_test, y_pred))
print('Classification Report:\n',classification_report(y_test,y_pred))

Accuracy:
 0.659217877094972
Confusion Matrix:
 [[94 14]
 [47 24]]
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.87      0.76       108
           1       0.63      0.34      0.44        71

    accuracy                           0.66       179
   macro avg       0.65      0.60      0.60       179
weighted avg       0.65      0.66      0.63       179



In [54]:
#5-Fold Cross Validation to generalise the model accuracy
kf=KFold(n_splits=5)
score=cross_val_score(svc,X,y,cv=kf)
print("Cross Validation Scores: {}".format(score))
print("Average Cross Validation score :{}".format(score.mean()))

Cross Validation Scores: [0.61452514 0.65730337 0.66292135 0.7247191  0.74719101]
Average Cross Validation score :0.6813319942250958


In [55]:
parameters ={'C': [0.1, 1, 10, 100, 1000],
             'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
             'kernel': ['rbf']}

In [56]:
grid_search = GridSearchCV( SVC(), 
                            parameters, 
                            cv=5,
                            scoring='accuracy',n_jobs=-1)
grid_result= grid_search.fit(X_train,y_train)
print('Best Params: ', grid_result.best_params_)
print('Best Score: ', grid_result.best_score_)

Best Params:  {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Best Score:  0.7008667388949079


In [57]:
grid_predictions = grid_search.predict(X_test)
print(confusion_matrix(y_test, grid_predictions))

[[99  9]
 [39 32]]


In [58]:
model_gridsearch = SVC(kernel='rbf', C=10, gamma=0.1)
model_gridsearch.fit(X_train,y_train)
y_pred = model_gridsearch.predict(X_test)
print('Accuracy Score:\n',accuracy_score(y_test, y_pred))
print('Confusion Matrix:\n',confusion_matrix(y_test, y_pred))
print('Classification Report:\n',classification_report(y_test,y_pred))

Accuracy Score:
 0.7318435754189944
Confusion Matrix:
 [[99  9]
 [39 32]]
Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.92      0.80       108
           1       0.78      0.45      0.57        71

    accuracy                           0.73       179
   macro avg       0.75      0.68      0.69       179
weighted avg       0.74      0.73      0.71       179



In [59]:
kf=KFold(n_splits=5)
score=cross_val_score(model_gridsearch,X,y,cv=kf)
print("Cross Validation Scores: {}".format(score))
print("Average Cross Validation score :{}".format(score.mean()))

Cross Validation Scores: [0.63687151 0.64606742 0.71910112 0.71348315 0.79213483]
Average Cross Validation score :0.7015316050467642


Random search

In [40]:
random_search=RandomizedSearchCV(estimator = SVC(), param_distributions=parameters,verbose=2, n_jobs=-1,
                            n_iter=50)
random_result = random_search.fit(X_train, y_train)
print('Best Score: ', random_result.best_score_*100)
print('Best Params: ', random_result.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Score:  70.0866738894908
Best Params:  {'kernel': 'rbf', 'gamma': 0.1, 'C': 10}


[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:    5.4s finished


In [41]:
model_randomsearch = SVC(kernel='rbf', C=10, gamma=0.1)
model_randomsearch.fit(X_train,y_train)
y_pred = model_randomsearch.predict(X_test)
print('Accuracy Score:\n',accuracy_score(y_test, y_pred))
print('Confusion Matrix:\n',confusion_matrix(y_test, y_pred))
print('Classification Report:\n',classification_report(y_test,y_pred))

Accuracy Score:
 0.7318435754189944
Confusion Matrix:
 [[99  9]
 [39 32]]
Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.92      0.80       108
           1       0.78      0.45      0.57        71

    accuracy                           0.73       179
   macro avg       0.75      0.68      0.69       179
weighted avg       0.74      0.73      0.71       179



In [42]:
kf=KFold(n_splits=5)
score=cross_val_score(model_randomsearch,X,y,cv=kf)
print("Cross Validation Scores: {}".format(score))
print("Average Cross Validation score :{}".format(score.mean()))

Cross Validation Scores: [0.63687151 0.64606742 0.71910112 0.71348315 0.79213483]
Average Cross Validation score :0.7015316050467642


 # SVM with parameter optimization using hyperopt

In [43]:
space = {"C": hp.choice("C", [0.1, 1, 10, 100, 1000]),
         "gamma": hp.choice("gamma", [1, 0.1, 0.01, 0.001, 0.0001]),
         "kernel": hp.choice("kernel", ['rbf']),}

In [44]:
# define objective function

def hyperparameter_tuning(params):
    model = SVC(**params)
    acc = cross_val_score(model, X_train, y_train,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [45]:
# Initialize trials object
trials = Trials()
best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=50, 
    trials=trials
)
print("Best: {}".format(best))

100%|███████████████████████████████████████████████| 50/50 [00:31<00:00,  1.61trial/s, best loss: -0.6924554318920516]
Best: {'C': 3, 'gamma': 1, 'kernel': 0}


In [46]:
model_hyperopt = SVC(kernel='rbf', C=1000, gamma=0.1)
model_hyperopt.fit(X_train,y_train)
y_pred = model_hyperopt.predict(X_test)
print('Accuracy Score:\n',accuracy_score(y_test, y_pred))
print('Confusion Matrix:\n',confusion_matrix(y_test, y_pred))
print('Classification Report:\n',classification_report(y_test,y_pred))

Accuracy Score:
 0.7653631284916201
Confusion Matrix:
 [[99  9]
 [33 38]]
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.92      0.83       108
           1       0.81      0.54      0.64        71

    accuracy                           0.77       179
   macro avg       0.78      0.73      0.73       179
weighted avg       0.77      0.77      0.75       179



In [47]:
kf=KFold(n_splits=5)
score=cross_val_score(model_hyperopt,X,y,cv=kf)
print("Cross Validation Scores are: {}\n".format(score))
print("Average of Cross Validation Scores: {}".format(score.mean()))

Cross Validation Scores are: [0.67039106 0.63483146 0.75280899 0.70224719 0.7752809 ]

Average of Cross Validation Scores: 0.7071119201556713


# Results

Cross Validation Score of SVM before hyperparameter tuning : 0.6480

Grid Search---->	C:10	Gamma:rbf	kernal:0.1	    Cross Validation Score:0.7032

Random Search-->	C:10	Gamma:rbf	kernal:0.1 	    Cross Validation Score:0.7015

HyperOpt------->	C:1000	Gamma:rbf	kernal:0.1	    Cross Validation Score:0.7071